In [53]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

In [54]:
# Path to the chromedriver executable
chromedriver_path = 'C:\Program Files\chromedriver-win64\chromedriver.exe'

# Set up the service
service = Service(chromedriver_path)

# Initialize the Chrome driver with the service
driver = webdriver.Chrome(service=service)

# Open the website
driver.get('https://www.notino.cz/zubni-pasty/')

# Unroll the website
while True:
    time.sleep(3)
    try:
        button = driver.find_element(By.XPATH, "//button[@data-testid='footer-action-button']")
        time.sleep(3)
        driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top - 100);", button)
        time.sleep(3)
        button.click()
    except NoSuchElementException:
        # Element not found, exit the loop
        break

# Initialize the DataFrame
notino = pd.DataFrame(columns=["Brand", "Name", "Description", "Price", "Discount", "Product URL", "Image"])  

# Find products
products = driver.find_elements(By.XPATH, "//div[@data-testid='product-container']")

# Scan products 
for product in products:
    # Sort out promo banners
    if (product.get_attribute('data-recommendation') != "promo-banner") and (product.get_attribute('data-recommendation') != "promo-banner-2"):
        # Find information
        brand = product.find_element(By.TAG_NAME, "h2").text
        name = product.find_element(By.TAG_NAME, "h3").text
        description = product.find_element(By.TAG_NAME, "p").text
        image = product.find_element(By.TAG_NAME, "img").get_attribute('src')
        prod_url = product.find_element(By.TAG_NAME, "a").get_attribute('href')
        try:
            price = product.find_element(By.XPATH, ".//span[@data-testid='price-component']").text
        except NoSuchElementException:
            price = np.NaN
        try:
            discount = product.find_element(By.CLASS_NAME, "styled__UnderlineWrapper-sc-1b3ggfp-0").text
        except NoSuchElementException:
            discount = ""
        # Fill the DataFrame
        notino = notino._append({"Brand":brand, "Name":name, "Description":description, "Price":price, "Discount":discount, "Product URL":prod_url, "Image":image}, ignore_index = True)

# Close the browser
driver.quit()

In [55]:
notino

,Brand,Name,Description,Price,Discount,Product URL,Image
0,Opalescence,Whitening,bělicí zubní pasta s fluoridem,99,,https://www.notino.cz/opalescence/original-for...,https://cdn.notinoimg.com/list_2k//opalescence...
1,Opalescence,Whitening Sensitivity Relief,bělicí pasta pro citlivé zuby,96,,https://www.notino.cz/opalescence/sensitivity-...,https://cdn.notinoimg.com/list_2k//opalescence...
2,GC,Tooth Mousse,remineralizační ochranný krém pro citlivé zuby...,437,,https://www.notino.cz/gc/tooth-mousse-tutti-fr...,https://cdn.notinoimg.com/list_2k//gc/28000119...
3,MEDIBLANC,Whitening,zubní pasta s bělicím účinkem,193,,https://www.notino.cz/mediblanc/whitening-zubn...,https://cdn.notinoimg.com/list_2k//mediblanc/3...
4,Elmex,Junior 6-12 Years,zubní pasta pro děti,94,,https://www.notino.cz/elmex/junior-6-12-years-...,https://cdn.notinoimg.com/list_2k//elmex/90030...
...,...,...,...,...,...,...,...
549,Chicco,Always Smiling Green,sada zubní péče Strawberry(pro děti),NaN,,https://www.notino.cz/chicco/always-smiling-gr...,https://cdn.notinoimg.com/list_2k//chicco/8058...
550,Biorepair,Advanced Senitivity,zubní pasta pro citlivé zuby pro ochranu zubů ...,NaN,,https://www.notino.cz/biorepair/advanced-senit...,https://cdn.notinoimg.com/list_2k//biorepair/8...
551,Colgate,Whitening,bělicí zubní pasta 75 ml,NaN,,https://www.notino.cz/colgate/whitening-belici...,https://cdn.notinoimg.com/list_2k//colgate/692...
552,BlanX,White Shock Instant White,bělicí zubní pasta 75 ml,119,,https://www.notino.cz/blanx/white-shock-belici...,https://cdn.notinoimg.com/list_2k//blanx/80173...


In [56]:
# Save the .csv
notino.to_csv("notino_raw.csv", index=False)